![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Stage 1: Install dependencies and setting up GPU environment

In [1]:
!pip install tensorflow-gpu==2.9.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Downloading the Dogs vs Cats dataset 

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2022-12-19 07:43:38--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 142.250.153.128, 173.194.79.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  29.8MB/s    in 2.2s    

2022-12-19 07:43:41 (29.8 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Stage 2: Dataset preprocessing

### Import project dependencies

In [4]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.9.2'

### Unzipping the Dogs vs Cats dataset

In [5]:
dataset_path = './cats_and_dogs_filtered.zip'

In [6]:
zip_object = zipfile.ZipFile(file=dataset_path, mode='r')

In [7]:
zip_object.extractall('./')

In [8]:
zip_object.close()

### Seting up dataset paths

In [9]:
dataset_path_new = './cats_and_dogs_filtered/'

In [10]:
train_dir = os.path.join(dataset_path_new, 'train')
validation_dir = os.path.join(dataset_path_new, 'validation')

## Building the model

### Loading the pre-trained model (MobileNetV2)

In [11]:
IMG_SHAPE = (128, 128, 3)

In [12]:
# include_top : decide whether fully connected network or not
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [13]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

### Freezing the base model

In [14]:
base_model.trainable = False

### Defining the custom head for our network

In [15]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [16]:
# because in this case, I would have 4*4*1280(20480) new weights to train. So, to reduce input shape I use GlobalAveragePooling
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [17]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [18]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Defining the model

In [19]:
# combine two networks(base_model, prediction_layer)
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

### Compiling the model

In [21]:
# Why the learning rate is so small?: because I used pretrain network.
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


### Creating Data Generators

Resizing images

  * Big pre-trained architecture support only certain input sizes.

For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [22]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [23]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 2000 images belonging to 2 classes.


In [24]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 1000 images belonging to 2 classes.


### Training the model

In [25]:
model.fit_generator(train_generator, epochs=50, validation_data=valid_generator)

<ipython-input-25-d90443073c77>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=50, validation_data=valid_generator)


Epoch 1/50
16/16 [==============================] - 15s 622ms/step - loss: 0.7001 - accuracy: 0.5995 - val_loss: 0.6589 - val_accuracy: 0.6320
Epoch 2/50
16/16 [==============================] - 8s 518ms/step - loss: 0.5925 - accuracy: 0.6920 - val_loss: 0.5734 - val_accuracy: 0.7160
Epoch 3/50
16/16 [==============================] - 8s 527ms/step - loss: 0.5166 - accuracy: 0.7625 - val_loss: 0.5049 - val_accuracy: 0.7820
Epoch 4/50
16/16 [==============================] - 8s 537ms/step - loss: 0.4535 - accuracy: 0.8115 - val_loss: 0.4474 - val_accuracy: 0.8240
Epoch 5/50
16/16 [==============================] - 8s 539ms/step - loss: 0.4028 - accuracy: 0.8435 - val_loss: 0.3996 - val_accuracy: 0.8520
Epoch 6/50
16/16 [==============================] - 8s 522ms/step - loss: 0.3599 - accuracy: 0.8650 - val_loss: 0.3590 - val_accuracy: 0.8690
Epoch 7/50
16/16 [==============================] - 8s 522ms/step - loss: 0.3226 - accuracy: 0.8865 - val_loss: 0.3246 - val_accuracy: 0.8840
Epoch

### Transfer learning model evaluation

In [26]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

<ipython-input-26-a86cc1f7f1e2>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)


In [27]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9629999995231628


## Fine tuning


There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 

### Un-freeze a few top layers from the model

In [28]:
base_model.trainable = True

In [29]:
print('Number of layersin the base modl: {}'.format(len(base_model.layers)))

Number of layersin the base modl: 154


In [30]:
# going to do fine tune all layers from 100 to 154
fine_tune_at = 100

In [31]:
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

### Compiling the model for fine-tuning

In [32]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Fine tuning

In [33]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

<ipython-input-33-f08455725960>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)


Epoch 1/5
16/16 [==============================] - 14s 588ms/step - loss: 0.1619 - accuracy: 0.9305 - val_loss: 0.0757 - val_accuracy: 0.9690
Epoch 2/5
16/16 [==============================] - 8s 532ms/step - loss: 0.0218 - accuracy: 0.9955 - val_loss: 0.0781 - val_accuracy: 0.9730
Epoch 3/5
16/16 [==============================] - 8s 530ms/step - loss: 0.0068 - accuracy: 0.9995 - val_loss: 0.0804 - val_accuracy: 0.9710
Epoch 4/5
16/16 [==============================] - 8s 534ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.0904 - val_accuracy: 0.9710
Epoch 5/5
16/16 [==============================] - 8s 543ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0955 - val_accuracy: 0.9730


### Evaluating the fine tuned model

In [34]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

<ipython-input-34-a86cc1f7f1e2>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)


In [35]:
print('Validation accuracy after fine tuning: {}'.format(valid_accuracy))

Validation accuracy after fine tuning: 0.9729999899864197
